In [ ]:
import os
import nltk
import csv

import math

import matplotlib.pyplot as plt


from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud

from bs4 import BeautifulSoup
from community import community_louvain
from fa2 import ForceAtlas2

In [ ]:
characters = []

# Get characters and communities, according to fandom graph
with open("communities_from_fandom.csv", "r", encoding="utf8") as sent_file:
    csv = csv.reader(sent_file, delimiter=",")
    for row in csv:
        name = row[0]
        community = row[1]
        characters.append((name, community))

In [ ]:
import csv
# Create an empty list to store tuples of words and their 
# average happiness score
sent_list = []

# Read in the .tsv file
with open("LabMT1.0.tsv", "r", encoding="utf8") as sent_file:
    tsv_reader = csv.DictReader(sent_file, delimiter="\t")
    # For each sentiment in the file, save the word and average happiness in a tuple
    # and add it to the list
    for sent in tsv_reader:
        word = sent["word"]
        average = sent["happiness_average"]
        sent_list.append((word, average))

## Calculating sentiment for characters by chapter

In [ ]:
# Defining function to calculate sentiment for a list of tokens
# Function for calculating the sentiment of a file from the frequency distribution for that file
def sentiment(tokens):
    # Total sentiment score of file
    sent_sum = 0.0
    # Total number of occurences of words
    occ_sum = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        for word, score in sent_list:
            if token == word:
                sent_sum += (float(score) * occ)
                occ_sum += occ
    return sent_sum / occ_sum

In [ ]:
# Finding sentiment from a concordance list
def con_sentiment(con_list):
    sent_sum = 0
    line_num = 0
    for item in con_list:
        # Make left and right into one list
        left = [ch.lower() for ch in item.left if ch.isalpha()]
        right = [ch.lower() for ch in item.right if ch.isalpha()]
        combined = left + right
        # Make freqDist, just in case
        fd = nltk.FreqDist(combined)
        sent_sum += sentiment(fd)
        line_num += 1
    return sent_sum / line_num

### Example start

In [ ]:
# Reading in the test text
f = open('test1.txt')
raw = f.read()
tokens = nltk.wordpunct_tokenize(raw)
text = nltk.Text(tokens)

In [ ]:
text.dispersion_plot(["Harry", "Dudley", "Mr", "Mrs", "Dursley", "Dumbledore", "Voldemort"])

In [ ]:
harry_con = text.concordance_list("Harry")
dursley_con = text.concordance_list("Dursley")
dumble_con = text.concordance_list("Dumbledore")
volde_con = text.concordance_list("Voldemort")

cons = [harry_con, dursley_con, dumble_con, volde_con]

# harry_con, type list
# harry_con[0], type ConcodanceLine
# harry_con[0].left, list of tokens appearing to the left
# harry_con[0].right, list of tokens appearing to the right

# nltk.FreqDist(harry_con[0].left)

In [ ]:
#sentiment(nltk.FreqDist(harry_con[0].left))
#print(harry_con[0].left)
fd = nltk.FreqDist(harry_con[0].left)
#sentiment(fd)
left = [ch.lower() for ch in harry_con[0].left if ch.isalpha()]
right = [ch.lower() for ch in harry_con[0].right if ch.isalpha()]
combined = left + right


In [ ]:
# Testing con_sentiment
for con in cons:
    print(con_sentiment(con))